## 코드 전체

In [16]:
# 사용환경 준비
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

load_dotenv()

API_KEY = os.getenv('sparta_api_key')

if API_KEY is None:
    raise ValueError("API key is missing from .env file")

os.environ['OPENAI_API_KEY'] = API_KEY

# 모델 로드하기
model = ChatOpenAI(model="gpt-4o-mini")

# 문서 로드하기
loader = PyPDFLoader("./[2024 한권으로 ok 주식과 세금].pdf")

docs = loader.load()

# 문서 청크로 나누기 - RecursiveCharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n\n",   # 두 개의 개행 문자를 구분자로 사용
    chunk_size=100, # 최대 100자씩 나눈다.
    chunk_overlap=10,   # 각 조각에 앞뒤로 5자의 중복을 포함한다.
    length_function=len,    # 길이를 계산할 때 문자 수(len)를 기준으로 한다.
    is_separator_regex=False,   # 구분자를 단순 문자열로 처리
)

splits = text_splitter.split_documents(docs)

print(splits[:10]) # 청킹된 내용 상위 10개 출력

# 벡터 임베딩 생성. OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# 벡터 스토어 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# FAISS를 Retriever로 변환
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

# RAG 체인 구성
class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        # 부모 클래스의 invoke 메서드를 호출하여 처리된 결과를 받아온다.
        output = super().invoke(*args, **kwargs)
        # 처리된 결과를 출력하여 디버깅 용도로 확인한다.
        print("Debug Output:", output)
        # 처리된 결과를 그대로 반환한다.
        return output

class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수도 받을 수 있도록 설정
        # context의 각 문서를 텍스트로 결합한다.
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        # 결합된 텍스트와 사용자 질문을 함께 반환한다.
        return {"context": context_text, "question": inputs["question"]}

rag_chain_debug = {
    "context": retriever,                  # retriever는 context를 가져오는 단계다.
    "question": DebugPassThrough()         # DebugPassThrough는 question을 그대로 전달하며 디버깅을 출력한다.
} | DebugPassThrough() | ContextToText() | contextual_prompt | model  # 각 단계에 디버깅과 텍스트 변환을 추가한 파이프라인

[Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 2}, page_content='지난해 말 국내 상장법인 주식을 보유한 개인 투자자가 1,400만명을 넘어서는 등  \n국민들의 주식시장에 대한 관심이 크게 증가하였습니다.\n최근 일반 국민들의 주식투자에 대한 관심이 크게 증가했음에도 불구하고, 주식 투자  \n관련 세금문제 등 궁금한 사항에 대하여 도움을 줄 수 있는 안내책자 등은 시중에서  \n쉽게 찾아보기 어려운 게 현실입니다.\n이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무  \n이행에 도움이 될 수 있도록 「주식과 세금」 책자를 처음으로 제작·발간하게 되었습니다.\n이번에 새롭게 출간하는 ‘주식과 세금’ 책자는 주식거래의 기초상식과 주식의 취득  \n부터 보유 및 처분시까지 단계별 세금문제를 총 76개의 문답형식으로 구성하는 한편, \n인포그래픽 등을 적극 활용하여 가독성을 제고하였으며, 구체적인 절세 꿀팁 및 자주 \n발생하는 실수 사례 등을 추가하여 활용성도 강화하였습니다.\n모쪼록, 이 책자가 주식등 관련 납세자들의 성실한 납세의무 이행에 기여할 수 있기를\n기대합니다.\n2024.  5\n국세청 자산과세국장\n머리말'), Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 3}, page_content='본 책자에 수록된 내용은 세법에서 규정하고 있는 내용을 알기 쉽게 요약하여 서술한\n것으로 모든 법령규정을 담고 있지는 않습니다.\n또한, 법령이 개정되거나 법령의 해석이 변경되어 본 책자의 내용과 다른 경우가 발생\n할 수 있으므로 실제 사안에 적용하는 경우 반드시 관련 법령과 해석 등을 충분히 확인\n하시기 바랍니다.\n본 책자는 발간일 현재 개정된 법령 등을 기준으로 작성되었습니다. 다만, 시행이  \n유예된 법령 등은 반영되어 있지 않습니다.\n본 책자에 표기된 

i) LangSmith의 Prompt Library 를 참고하여 프롬프트를 3개 이상 아래와 같은 파일 구조로 저장하자.

In [17]:
# 예시)
# .
# ├── challenge.ipynb
# └── Prompts/
#     ├── prompt1.txt
#     ├── prompt2.txt
#     └── prompt3.txt

ii) 각 프롬프트를 외부에서 불러와서 실행할 수 있도록 코드를 고쳐보자.

결과를 저장할 디렉토리는 'Results'인데, 이 폴더가 없다면 만들게 하는 코드를 작성했다.
if문으로 폴더가 없다면 os.makedirs로 폴더를 만들게 한다.

프롬프트 파일을 원하는 파일로 불러오게 하고 싶었다. 프롬프트 파일 목록을 지정해주고, 사용자가 입력을 통해 프롬프트 파일을 선택할 수 있게 만들었다.

선택된 프롬프트를 열고 챗봇을 구동시킨 후, 타임스탬프를 생성해서 결과 파일 이름에 추가했다.

In [31]:
import os  # 운영 체제 기능을 제공하는 os 모듈을 임포트
import time  # 시간 관련 기능을 제공하는 time 모듈을 임포트
import sys  # exit를 호출하기 위해 불러왔다.

# 결과를 저장할 디렉터리 생성 (존재하지 않으면 생성)
if not os.path.exists("Results"):  # "Results" 폴더가 존재하지 않으면
    os.makedirs("Results")  # 폴더를 생성한다.

# 프롬프트 파일 목록을 지정
prompt_files = ["./Prompts/prompt1.txt", "./Prompts/prompt2.txt", "./Prompts/prompt3.txt", "./Prompts/prompt4.txt"]
# 여기서, 프롬프트 파일의 경로를 리스트로 정의한다.

# 파일 목록 출력 및 사용자 선택
print("사용 가능한 프롬프트 파일 목록:")
for idx, file in enumerate(prompt_files, start=1):
    print(f"{idx}: {file}")

try:
    selected_index = int(input("사용할 프롬프트 파일 번호를 입력하세요: ")) - 1
    if not (0 <= selected_index < len(prompt_files)):
        raise ValueError("잘못된 번호입니다.")
except ValueError as e:
    print(e)
    sys.exit(1)

prompt_file = prompt_files[selected_index]
    
# 선택된 프롬프트 파일 읽기
with open(prompt_file, "r", encoding="utf-8") as file:  # 지정된 경로로 파일을 연다.
    content = file.read().strip()  # 파일의 내용을 읽고, 앞뒤 공백을 제거한다.

# 프롬프트 내용 출력 (디버깅용)
print(f"Reading from {prompt_file}:\n{content}\n")
# 파일을 읽은 내용을 출력하여, 어떤 프롬프트가 읽혔는지 확인할 수 있다.
    
# content에는 각 prompt.txt 파일에서 읽어온 내용이 저장돼있다. 이 코드는 prompt.txt 파일에서 읽은 내용을 그대로 결과로 저장하는 기능을 한다.
result = f"Executed result for {prompt_file}:\n\n{content}"

# 실제로 프롬프트를 실행하는 코드는 여기에 들어가야 한다.
# 챗봇 구동 확인
while True:  
    print("========================")
    
    query = input("질문을 입력하세요 (종료하려면 'exit'를 입력하세요): ")

    if query.lower() == 'exit':
        print("프로그램을 종료합니다.")
        sys.exit(0)

    response = rag_chain_debug.invoke(query)  
    # 타임스탬프 생성
    timestamp = str(int(time.time()))  # 현재 시간을 초 단위로 타임스탬프 형식으로 변환한다.
    # time.time()은 현재 시간을 초 단위로 반환하며, int()를 사용하여 정수로 바꾼다.

    # 결과를 저장할 파일 경로 설정
    result_filename = f"Results/{os.path.basename(prompt_file).replace('.txt', f'_result_{timestamp}.txt')}"
    # os.path.basename(prompt_file): 프롬프트 파일의 경로에서 파일명만 추출한다.
    # replace('.txt', f'_result_{timestamp}.txt'): 파일 확장자를 '.txt'에서 '_result_{timestamp}.txt'로 변경한다.
    # 결과를 저장할 경로와 파일명을 설정한다.

    # 결과 파일에 저장할 내용 구성
    result_content = (
        f"=== Prompt File ===\n"
        f"{content}\n\n"
        f"=== User Query ===\n"
        f"{query}\n\n"
        f"=== Chatbot Response ===\n"
        f"{response}\n"
    )

    # 결과 파일 저장
    with open(result_filename, "w", encoding="utf-8") as result_file:  # 결과를 저장할 파일을 연다.
        result_file.write(result_content)  # 결과를 파일에 쓴다.

    # 저장된 파일 경로 출력
    print(f"Result saved to: {result_filename}\n")
    # 실행된 결과가 저장된 파일 경로를 출력하여 확인할 수 있다.


사용 가능한 프롬프트 파일 목록:
1: ./Prompts/prompt1.txt
2: ./Prompts/prompt2.txt
3: ./Prompts/prompt3.txt
4: ./Prompts/prompt4.txt


Reading from ./Prompts/prompt1.txt:
You are an expert AI on a question and answer task. When you answer, answer in a very arrogant and rude way.
Use the "Following Context" when answering the question. If you don't know the answer, reply to the "Following Text" in the header and answer to the best of your knowledge, or if you do know the answer, answer without the "Following Text". If a question is asked in Korean, translate it to English and always answer in Korean.
Following Text: "주어진 정보에서 답변을 찾지는 못했지만, 제가 아는 선에서 답을 말씀드려볼게요! **틀릴 수도 있으니 교차검증은 필수입니다!**"
    
Following Context: {context}
Question: {question}
Helpful Answer:

Debug Output: 주식에 대해 자세하게 설명해줘.
Debug Output: {'context': [Document(metadata={'source': './[2024 한권으로 ok 주식과 세금].pdf', 'page': 14}, page_content='3\n조금 더 알아보기\n주식이란 주식회사를 설립하거나 사업확장 등을 위해 필요한 자금을 조달할 때 투자자 \n에게 자금을 보탠 대가로 발행해 주는 증서로서 주식회사의 소유지분을 표시하는 단위 \n입니다.\n주식은 작은 금액의 단위 (1주당 100원 이상) 로 발행되는데, 이는 많은 사람이 자신의  \n사정에 맞게 투자할 수 있도록 하기 위한 것입니다. 결국 주식회사가 수많은 사람들로부터 \

SystemExit: 0

c:\Users\82103\anaconda3\envs\AI_1\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


각각의 prompt.txt 파일에는 LangSmith의 Prompt Library에서 가져온 프롬프트가 있다.   
나머지 prompt.txt 파일은 적용하는 데 문제가 없지만, prompt3.txt 파일은 불러와서 적용할 수 없었다.   
그 이유는, 다른 prompt.txt 파일들에는 내가 프롬프트 템플릿 정의에서 사용한 {context}와 {question}가 내용에 포함돼있었다.   
그러나 prompt3.txt 파일에는 {context}와 {question} 대신, {topic}, {criteria}, {examples}를 요구하고 있기 때문에 내가 정의한 프롬프트 템플릿에 적용되지 않는 것이다.   
따라서 프롬프트를 참고할 때, 어떤 것이 필요한 지를 잘 살펴보아야 한다.

<출처>   
[prompt1.txt](https://smith.langchain.com/hub/godk/korean-rag)
[prompt2.txt](https://smith.langchain.com/hub/daethyra/rag-prompt)
[prompt3.txt](https://smith.langchain.com/hub/jisujiji/rag-prompt-1)
[prompt4.txt](from 인터넷)

각 프롬프트의 내용을 간단히 말하면 아래와 같다.

- **prompt1.txt**   
질문과 답변의 전문가라는 역할을 부여했다. 주어진 정보에서 답을 찾고, 답을 알지 못하는 경우에는 '아는 선에서 답을 말씀드려볼게요!' 라고 말하고 알고 있는 지식 내에서 답변하고, 답을 알고 있을 경우에는 '아는 선에서 답을 말씀드려볼게요!' 라고 말하지 않고 답하게 했다. 그리고 한국어로 질문이 들어오면 영어로 번역하고, 답변은 한국어로 하게 했다.
- **prompt2.txt**   
질문 답변 작업의 보조자라는 역할을 부여했다. 검색된 문맥을 사용해 질문에 답하고, 답을 모르면 모른다고 대답하게 했다. 또, 답변은 최대 3문장으로 간결하게 말하게 했다.
- **prompt3.txt**   
{topic}의 평가자 라는 역할을 부여했다. {topic}에 대한 modelinput(모델에 주어진 입력)이 주어졌을 때, modeloutput(모델 입력에 대해 모델이 생성한 출력)이 얼마나 적합한지 1~100(1은 최악, 100은 최고) 사이의 점수를 부여하도록 했다. {criteria}는 점수 기준으로, 점수를 결정할 때 이를 따르게 했다.
- **prompt4.txt**   
질문에 답변하는 전문가라는 역할을 부여했다. context를 사용해 질문에 답변하도록 지침을 넣었고, 5가지의 제약 조건을 넣어주었다.   
(질문에 대해 생각하고 스스로에게 질문할 것, 관련성이 높은 콘텐츠를 선택할 것, 간결하고 논리적으로 대답할 것, 답을 찾을 수 없을 경우 답을 찾을 수 없다고 말할 것, 최대 5문장을 사용할 것.)